# Voxelization

[More about voxelization](/spatial_computing_project_template/A2_Configuring/Process/#building-envelope)

In this notebook, we will voxelize our envelope. Because of our high resolution voxelsize of 3.6m we are using interpolation with a low resolution voxelsize of 10.8m. We are using the outputs of this notebook for the analysis.

The inputs of this notebook are the envelope as an obj file.
We went over this notebook twice. The first time we specified the voxelsize as 3.6, the second time as 10.8

### 0. Initialization

In [ ]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np

In [ ]:
vs = 3.6
unit = [vs, vs, vs]
mesh_path = os.path.relpath('../data/envelope_new.obj')
context_path = os.path.relpath('../data/immediate_context.obj')
context_mesh = tm.load(context_path)


## 1. Input Mesh

In [ ]:
# load the mesh from file
mesh = tm.load(mesh_path)
# Check if the mesh is watertight
print(mesh.is_watertight)

In [ ]:
# convert trimesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')


# adding the base mesh: light blue
p.add_mesh(tri_to_pv(mesh), color='#abd8ff')

# plotting
p.show(use_ipyvtk=True)

In [ ]:
mesh.vertices += np.array([0,0,-0.01])

## 2. Voxelize the Mesh

In [ ]:
# initialize the base lattice
base_lattice = tg.lattice(mesh.bounds, unit=unit, default_value=1, dtype=int)

# check which voxel centroids is inside the mesh
interior_condition = mesh.contains(base_lattice.centroids)

# reshape the interior condition to the shape of the base_lattice
interior_array = interior_condition.reshape(base_lattice.shape)

# convert the interior array into a lattice
interior_lattice = tg.to_lattice(interior_array, base_lattice.minbound, base_lattice.unit)


In [ ]:
# convert trimesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

p.add_mesh(tri_to_pv(context_mesh), style='surface')

# fast visualization of the lattice
interior_lattice.fast_vis(p)

# adding the base mesh: light blue
p.add_mesh(tri_to_pv(mesh), color='#abd8ff', opacity=0.1)

# plotting
p.show(use_ipyvtk=True)


## 3. Saving the lattice to CSV

In [ ]:
csv_path = os.path.relpath('../data/envelope_highres_new.csv')
interior_lattice.to_csv(csv_path)

In [ ]:
vox_cens = interior_lattice.centroids
vox_count = len(vox_cens)
print(vox_count)

### Credits

In [ ]:
__author__ = "Shervin Azadi"
__changes_made_by__ = "Lotte Zwolsman"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/frankvahstal/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on Voxelization"